In [10]:
# ==============================================================================
# Шаг 1: Установка и импорт зависимостей
# ==============================================================================

import pandas as pd
import torch
import re
from transformers import AutoTokenizer, AutoModelForTokenClassification

In [3]:
# ==============================================================================
# ШАГ 3 (ОБНОВЛЕННЫЙ И ОЧИЩЕННЫЙ)
# Загрузка модели и определение ВСЕХ необходимых функций
# ==============================================================================

import pandas as pd
import torch
import re
from transformers import AutoTokenizer, AutoModelForTokenClassification

# --- Загрузка модели ---
MODEL_DIR = "./rubert_tiny_spaced_restoration_best_f1"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
    model = AutoModelForTokenClassification.from_pretrained("FacebookAI/xlm-roberta-base")
    model.to(device)
    model.eval()
    print("✅ Обученная модель и токенизатор успешно загружены.")
except OSError:
    print(f"❌ ОШИБКА: Директория с моделью '{MODEL_DIR}' не найдена!")


# --- Основная функция предсказания ---
def predict_all(text: str) -> tuple[str, list[int]]:
    """
    Обрабатывает текст и возвращает кортеж из двух элементов:
    1. Текст с восстановленными пробелами.
    2. Список индексов, в которые нужно вставить пробел.
    """
    # ЭТА ПРОВЕРКА ИСПРАВЛЯЕТ ОШИБКУ INDEXERROR
    if not isinstance(text, str) or not text:
        return "", []

    text_lower = text.lower()
    
    inputs = tokenizer(text_lower, return_tensors="pt", truncation=True, is_split_into_words=False)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        logits = model(**inputs).logits
    
    predictions = torch.argmax(logits, dim=2)[0].cpu().numpy()
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0].cpu().numpy())

    restored_text = ""
    space_indices = []
    current_char_index = 0
    is_first_token = True

    for token, prediction in zip(tokens, predictions):
        if token in (tokenizer.cls_token, tokenizer.sep_token, tokenizer.pad_token):
            continue
        
        if prediction == 1 and not is_first_token:
            restored_text += " "
            space_indices.append(current_char_index)
            
        cleaned_token = token.replace('##', '')
        restored_text += cleaned_token
        current_char_index += len(cleaned_token)
        is_first_token = False

    return restored_text, space_indices

print("✅ Универсальная функция 'predict_all' готова.")


# --- Функция пост-обработки пунктуации ---
def post_process_punctuation(text: str) -> str:
    """
    Выполняет интеллектуальную финальную чистку текста:
    1. Различает тире и дефис, используя словарь исключений для сложных слов.
    2. Добавляет пробел после запятой, если его нет.
    """
    COMMON_HYPHENATED_WORDS = [
        'санкт-петербург', 'бизнес-ланч', 'пресс-конференция', 'онлайн-урок',
        'веб-сайт', 'серо-буро-малиновый', 'когда-нибудь', 'где-либо',
        'ростов-на-дону', 'кто-то'
    ]
    HYPHEN_PARTICLES_PREFIX = ['по', 'кое']
    HYPHEN_PARTICLES_SUFFIX = ['то', 'либо', 'нибудь', 'ка', 'таки', 'де']

    processed_text = re.sub(r'\s*,\s*', ', ', text)
    processed_text = re.sub(r'\s*-\s*', ' - ', processed_text)

    for word in COMMON_HYPHENATED_WORDS:
        spaced_word = word.replace('-', ' - ')
        processed_text = re.sub(spaced_word, word, processed_text, flags=re.IGNORECASE)

    for suffix in HYPHEN_PARTICLES_SUFFIX:
        pattern = re.compile(rf'(\s-\s)({suffix})\b', re.IGNORECASE)
        processed_text = pattern.sub(rf'-\2', processed_text)
    
    for prefix in HYPHEN_PARTICLES_PREFIX:
        pattern = re.compile(rf'\b({prefix})(\s-\s)', re.IGNORECASE)
        processed_text = pattern.sub(rf'\1-', processed_text)
        
    return processed_text
    
print("✅ Интеллектуальная функция 'post_process_punctuation' готова.")


Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Обученная модель и токенизатор успешно загружены.
✅ Универсальная функция 'predict_all' готова.
✅ Интеллектуальная функция 'post_process_punctuation' готова.


In [136]:
# ==============================================================================
# Шаг 7: Сохранение финальной модели на диск
# ==============================================================================

# Директория для сохранения (должна совпадать с той, что в конфигурации)
OUTPUT_MODEL_DIR = "./DeepPavlov/rubert-base-cased_spaced_restoration_20_epoch_16_batch_0.1_0.00002"

print(f"💾Сохранение финальной модели в директорию: '{OUTPUT_MODEL_DIR}'")

# Используем `os` для надежного создания директории, если ее нет.
# А также `shutil` для удаления старой версии, чтобы избежать конфликтов.
import os
import shutil

if os.path.exists(OUTPUT_MODEL_DIR):
    print("  -> Найдена старая версия модели. Удаляем...")
    shutil.rmtree(OUTPUT_MODEL_DIR)

print(f"  -> Создаем новую директорию '{OUTPUT_MODEL_DIR}'...")
os.makedirs(OUTPUT_MODEL_DIR, exist_ok=True)

# Сохраняем модель. Метод `save_pretrained` сохранит и веса (pytorch_model.bin),
# и конфигурационный файл (config.json).
model.save_pretrained(OUTPUT_MODEL_DIR)

# Сохраняем токенизатор. Это не менее важно! Он сохранит свой словарь
# (vocab.txt) и конфигурационные файлы.
tokenizer.save_pretrained(OUTPUT_MODEL_DIR)

print(f"\n✅ Готово! Модель и токенизатор сохранены в '{OUTPUT_MODEL_DIR}' и готовы к использованию в другом ноутбуке.")


💾Сохранение финальной модели в директорию: './DeepPavlov/rubert-base-cased_spaced_restoration_20_epoch_16_batch_0.1_0.00002'
  -> Найдена старая версия модели. Удаляем...
  -> Создаем новую директорию './DeepPavlov/rubert-base-cased_spaced_restoration_20_epoch_16_batch_0.1_0.00002'...

✅ Готово! Модель и токенизатор сохранены в './DeepPavlov/rubert-base-cased_spaced_restoration_20_epoch_16_batch_0.1_0.00002' и готовы к использованию в другом ноутбуке.


In [ ]:
# ==============================================================================
# Шаг 4: Запуск тестирования, пост-обработка и создание DataFrame
# ==============================================================================

TEST_FILE_NAME = "text_data.txt"


try:
    # Списки для хранения всех необходимых данных
    ids_list = []
    original_texts_list = []
    restored_texts_list = []
    predicted_indices_list = []

    # Читаем тестовый файл построчно
    with open(TEST_FILE_NAME, 'r', encoding='utf-8') as f:
        # Пропускаем заголовок, если он есть
        next(f, None)
        for line in f:
            line = line.strip()
            if not line:
                continue

            try:
                # Разделяем строку по первой запятой на 'id' и 'текст'
                line_id, text_no_spaces = line.split(',', 1)

                # 1. Получаем предсказания от нейросети
                restored_text_from_model, space_indices = predict_all(text_no_spaces)
                # 2. ПРИМЕНЯЕМ ПОСТ-ОБРАБОТКУ
                final_restored_text = post_process_punctuation(restored_text_from_model)

                # Добавляем все данные в соответствующие списки
                ids_list.append(line_id)
                original_texts_list.append(text_no_spaces)
                restored_texts_list.append(final_restored_text)
                predicted_indices_list.append(space_indices)

            except ValueError:
                print(f"⚠️ ПРЕДУПРЕЖДЕНИЕ: Строка '{line}' не соответствует формату 'id,text' и будет пропущена.")

    # Создаем итоговый DataFrame из списков
    if ids_list:
        results_df = pd.DataFrame({
            'id': ids_list,
            'text_no_spaces': original_texts_list,
            'restored_text': restored_texts_list,
            'predicted_positions': predicted_indices_list
        })
        
        print("\n✅ DataFrame с результатами (включая интеллектуальную пост-обработку) успешно создан:")
        
        pd.set_option('display.max_colwidth', 120)
        display(results_df)

    else:
        print("✅ Тестирование завершено. В файле не найдено данных для обработки.")


except NameError as e:
    if 'post_process_punctuation' in str(e):
         print("❌ ОШИБКА: Функция 'post_process_punctuation' не определена. Убедитесь, что вы выполнили ячейку с её определением.")
    elif 'predict_all' in str(e):
        print("❌ ОШИБКА: Функция 'predict_all' не определена. Убедитесь, что вы выполнили ячейку с её определением.")
    else:
        print(f"❌ ОШИБКА: {e}. Пожалуйста, убедитесь, что все предыдущие ячейки выполнены.")
except FileNotFoundError:
    print(f"❌ ОШИБКА: Файл '{TEST_FILE_NAME}' не найден. Пожалуйста, убедитесь, что он существует.")



✅ DataFrame с результатами (включая интеллектуальную пост-обработку) успешно создан:


,id,text_no_spaces,restored_text,predicted_positions
0,0,куплюайфон14про,куплю айфон 14про,"[5, 10]"
1,1,ищудомвПодмосковье,ищудомвпод москов ье,"[10, 16]"
2,2,сдаюквартирусмебельюитехникой,сда юквартир усмебель юитехникой,"[3, 11, 19]"
3,3,новыйдивандоставканедорого,новыйдивандоставкан ед орого,"[19, 21]"
4,4,отдамдаромкошку,отдамдаром кошку,[10]
...,...,...,...,...
1000,1000,Янеусну.,янеу сну .,"[4, 7]"
1001,1001,Весна-яуженегреюпио.,весна - яуженегре юпи о .,"[15, 18, 19]"
1002,1002,Весна-скоровырастеттрава.,весна - скоровы растетт рава .,"[13, 20, 24]"
1003,1003,"Весна-выпосмотрите,каккрасиво.","весна - выпосмотр ите, как крас иво .","[6, 15, 19, 22, 26, 29]"


In [5]:
results_df = results_df[['id', 'predicted_positions']]

In [133]:
results_df.to_csv("submission.csv")

In [7]:
results_df[150:200]

,id,predicted_positions
150,150,[25]
151,151,[]
152,152,[]
153,153,[]
154,154,[6]
155,155,[]
156,156,"[9, 17, 19]"
157,157,"[3, 14]"
158,158,"[10, 12]"
159,159,[22]
